In [1]:
%reload_ext autoreload
%autoreload 2

import cv2

import os
import sys
sys.path.append(os.path.join(os.environ['REPO_DIR'], 'utilities'))
from utilities2015 import *

from collections import defaultdict

from matplotlib.path import Path
%matplotlib inline

import pandas as pd

from joblib import Parallel, delayed

In [2]:
scoremaps_rootdir = '/oasis/projects/nsf/csd395/yuncong/CSHL_scoremaps_lossless'

In [3]:
scoremapViz_rootdir = '/oasis/projects/nsf/csd395/yuncong/CSHL_scoremapViz'
if not os.path.exists(scoremapViz_rootdir):
    os.makedirs(scoremapViz_rootdir)

In [4]:
labels_to_detect = ['5N', '7n', '7N', '12N', 'Gr', 'LVe', 'Pn', 'SuVe', 'VLL']

In [5]:
downscale_factor = 8

In [10]:
stack = 'MD594'

# for sec in range(first_bs_sec, last_bs_sec+1):
# for sec in range(157, 166+1):
for sec in range(166, 200):
# for sec in range(266, 303+1):
# for sec in [161]:

    print sec
    
    scoremapViz_dir = os.path.join(scoremapViz_rootdir, stack, '%04d'%sec)
    if not os.path.exists(scoremapViz_dir):
        os.makedirs(scoremapViz_dir)

    dm = DataManager(stack=stack, section=sec)
    dm._load_image(['rgb-jpg'])

    dataset = '%(stack)s_%(sec)04d_roi1' % {'stack': stack, 'sec': sec}

    for l in labels_to_detect:
        print l
        
        scoremap_bp_filepath = scoremaps_rootdir + '/%(stack)s/%(slice)04d/%(stack)s_%(slice)04d_roi1_denseScoreMapLossless_%(label)s.bp' \
            % {'stack': stack, 'slice': sec, 'label': l}
            
        if not os.path.exists(scoremap_bp_filepath):
            sys.stderr.write('No scoremap for section %d\n' % sec)
            continue
    
        dense_score_map_lossless = bp.unpack_ndarray_file(scoremap_bp_filepath)

#         dense_score_map_lossless = np.load(os.path.join(scoremaps_rootdir, stack, '%04d'%sec,
#                                                         dataset + '_denseScoreMapLossless_%s.npz' % l))['arr_0']
        scoremap_viz = plt.cm.hot(dense_score_map_lossless[::downscale_factor, ::downscale_factor])

        viz = alpha_blending(scoremap_viz[..., :3], dm.image_rgb_jpg[::downscale_factor, ::downscale_factor], .3, 1.)
        
        cv2.putText(viz, l, (50, 50), cv2.FONT_HERSHEY_DUPLEX, 2, ((0,0,0)), 3)
        
        cv2.imwrite(os.path.join(scoremapViz_dir, dataset+'_scoremapViz_%s.jpg' % l), 
                    img_as_ubyte(viz[..., [2,1,0]]))
        

166
5N


ValueError: operands could not be broadcast together with shapes (1448,1804) (181,226) 